# Election Tweets: Geolocation

To understand just how real the "red state/blue state" divide is on the tweet level, I need some geolocation data. Since only very few users (<2%) activate the geotagging feature on their user accounts, I'll need to get this information from other sources. I decided to extract this info from people's self-reported location (in their user profile), but now I want to see what the data looks like and what information I can extract to put towards modelling.

In [40]:
import pandas as pd

import warnings
warnings.filterwarnings('ignore')

In [41]:
tweets = pd.read_csv('/Users/laraehrenhofer/Documents/Coding_Projects/git_repos/tweet-the-people-legacy/data/tweet_pg.csv')

## Step 1: Understanding geolocation by numbers

Some introductory questions:
- How many tweets have geotagged location vs. location based on profile info?
- Where are most tweets located?
- Any differences by Republican/Democrat ticket?

### Geotagged vs. profile geolocation

Some basic cleanup is necessary as some of these locations were generated in development versions of the tweet streaming script.

In [134]:
loc_no = pd.DataFrame(tweets.groupby(['loc_type', 'ticket']).count()['tweet_id']).unstack()

In [135]:
keep_rows = ['bound_box_coords', 'no_loc', 'user_loc']
drop_rows = [column for column in loc_no.index.values if column not in keep_rows]

In [136]:
loc_no = loc_no.drop(drop_rows, axis=0)
loc_no = pd.DataFrame(loc_no.to_records())
loc_no.columns = ['loc_type', 'Democrat', 'Republican']
loc_no['total'] = loc_no['Democrat'] + loc_no['Republican']

In [137]:
def get_ticket_percent(data, ticket):
    colname_total = f'percent_of_total_{ticket}'
    colname_ticket = f'percent_of_ticket_{ticket}'
    data[colname_total] = data[ticket].apply(lambda x: round((x/sum(data['total']))*100, 2))
    data[colname_ticket] = data[ticket].apply(lambda x: round((x/sum(data[ticket]))*100, 2))
    return data

In [138]:
tickets = ['Democrat', 'Republican']

for ticket in tickets:
    loc_no = get_ticket_percent(loc_no, ticket)

In [142]:
loc_no['total_percent'] = loc_no['total'].apply(lambda x: round((x/sum(loc_no['total']))*100, 2))

In [143]:
loc_no

,loc_type,Democrat,Republican,total,percent_of_total_Democrat,percent_of_ticket_Democrat,percent_of_total_Republican,percent_of_ticket_Republican,total_percent
0,bound_box_coords,851.0,401.0,1252.0,0.27,0.45,0.13,0.31,0.40
1,no_loc,78283.0,50747.0,129030.0,24.75,41.53,16.04,39.70,40.79
2,user_loc,109385.0,76665.0,186050.0,34.58,58.02,24.24,59.98,58.81


#### Conclusions

1. Less than half a percent of this tweet database has geotagged location data.
2. 40% has no location at all.
3. Between geotagging and user profile location data, we still have close to 60% of the dataset available.
4. The fact of there being more Democrat than Republican tweets overall makes it initially look like there's a partisan divide in terms of people's provision of geolocation data (looking at the `percent_of_total` columns). However, breaking it down into percent by ticket suggests that there isn't an enormous contrast between the two populations after all. (Could determine further statistical information using a logistic regression here but it doesn't seem interesting enough to warrant the extra attention.)

### Tweet locations by state